In [1]:
#Bonus: What topics do the wine descriptions fall under? (Hint: Use a topic modeling algorithm like Latent Dirichlet
#allocation on the description column

              #######################          LIBRARIES                 #######################

#Import required libraries to perform topic modeling on wine dataset
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
#from nltk.stem.porter import *
import numpy as np
#np.random.seed(2018)
#import nltk
#nltk.download('wordnet')
import pandas as pd
from nltk.stem import SnowballStemmer

/anaconda3/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
              
           #######################          DATAFRAME PREPARATION    #######################

#Read the csv file as a dataframe
wine_df = pd.read_csv("/Users/atyam/Desktop/M&M/winemag-data-130k-v2.csv")
#wine_df.head()

In [3]:
#Drop the duplicates from the dataframe.
wine_df.drop_duplicates(subset=['description'], keep="first", inplace = True)

In [4]:
#Assign the wine description value to new variable for further analysis
wine_text = wine_df[['description']]

In [6]:
#Insert an index column into the new dataframe for indexing few rows to easlity analyze the data. 
wine_text['index'] = wine_text.index

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
#documents = data_text

In [7]:
print(len(wine_text))
print(wine_text[:5])

119955
                                         description  index
0  Aromas include tropical fruit, broom, brimston...      0
1  This is ripe and fruity, a wine that is smooth...      1
2  Tart and snappy, the flavors of lime flesh and...      2
3  Pineapple rind, lemon pith and orange blossom ...      3
4  Much like the regular bottling from 2012, this...      4


In [20]:
            
             #####################         TOPIC MODELING ANALYSIS               ###################
        

In [8]:
#Assign the stemmer method to a variable to apply on further analysis
snb = SnowballStemmer('english')

In [9]:
# Defined a function to perform lemmatize and stem preprocessing steps on the dataset.
def lem_stemming(text):
    return snb.stem(WordNetLemmatizer().lemmatize(text))            #Pos= default i.e is noun
def preprocess(text):
    text_tokens = []                                               # array of tokens are returned after preprocessing
    for token in gensim.utils.simple_preprocess(text):  
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            text_tokens.append(lem_stemming(token))
    return text_tokens

In [ ]:
#Select a single row from the wine_text by indexing to preview after the results of preprocessing the texts. 

In [10]:
wine_doc_sample = wine_text[wine_text['index'] == 250].values[0][0]
print('Actual Texts: ')
words = []
for word in wine_doc_sample.split(' '):   # took a sample of one row/index and split that line to str/words and then append to a list.
    words.append(word)
print(words)
print('\n\n Tokenized and lemmatized Texts: ')
print(preprocess(wine_doc_sample))

Actual Texts: 
['Ripe,', 'soft', 'and', 'a', 'little', 'sweet,', 'with', 'jellied', 'flavors', 'of', 'blackberries,', 'cherries,', 'chocolate', 'and', 'anise.', 'Drink', 'now.']


 Tokenized and lemmatized Texts: 
['ripe', 'soft', 'littl', 'sweet', 'jelli', 'flavor', 'blackberri', 'cherri', 'chocol', 'anis', 'drink']


In [12]:
# The preprocess function/method is invoked to the description column of the dataframe by mapping.
processed_text = wine_text['description'].map(preprocess)  

# Pint first 5 output of lemmatization after preprocessing.
processed_text[:5]                        

0    [aroma, includ, tropic, fruit, broom, brimston...
1    [ripe, fruiti, wine, smooth, structur, firm, t...
2    [tart, snappi, flavor, lime, flesh, rind, domi...
3    [pineappl, rind, lemon, pith, orang, blossom, ...
4    [like, regular, bottl, come, rough, tannic, ru...
Name: description, dtype: object

In [ ]:
         
               ######################  BAG OF WORDS ON DATASET      ########################
    
    
#Create a dictionary from ‘processed_text’ containing the number of times a word appears;i.e to convert texts to 
#vectors. Here, each word is represented by one vector i and the number of times the ith word appears in the document.

In [13]:
dictionary = gensim.corpora.Dictionary(processed_text) 

In [14]:
#Here there are 20941 distinct woords in the processed corpus
print(dictionary)

Dictionary(20941 unique tokens: ['acid', 'alongsid', 'appl', 'aroma', 'brimston']...)


In [ ]:
                ##########3#############        GGENSIM doc2bow            #######################3

#Now use doc2bow to count the number of occurances for each distinct word. It conoverts the woord to its integer word id 
#and returns in the form of (word id, word ocunt)

In [17]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_text]

# indexed a row for sample analysis
print(bow_corpus[250])

#The above process is elaborated further
bow_doc_250 = bow_corpus[250]
for i in range(len(bow_doc_250)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_250[i][0], 
                                               dictionary[bow_doc_250[i][0]], 
bow_doc_250[i][1]))

[(28, 1), (36, 1), (84, 1), (109, 1), (125, 1), (132, 1), (155, 1), (315, 1), (332, 1), (495, 1), (779, 1)]
Word 28 ("ripe") appears 1 time.
Word 36 ("flavor") appears 1 time.
Word 84 ("blackberri") appears 1 time.
Word 109 ("soft") appears 1 time.
Word 125 ("drink") appears 1 time.
Word 132 ("chocol") appears 1 time.
Word 155 ("cherri") appears 1 time.
Word 315 ("anis") appears 1 time.
Word 332 ("sweet") appears 1 time.
Word 495 ("jelli") appears 1 time.
Word 779 ("littl") appears 1 time.


In [ ]:
       #######################       PERFORMING LDA USING BAG OF WORDS       ##########################
    


In [18]:
#Train the lda model using gensim.models.LdaMulticore, assumed values are assumed for passes and iterations.
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, iterations =50)

In [ ]:
#For each topic, we will explore the words occuring in that topic and its relative weight.

In [19]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.028*"wine" + 0.019*"vineyard" + 0.017*"light" + 0.015*"pinot" + 0.015*"flavor" + 0.015*"acid" + 0.013*"grape" + 0.012*"blend" + 0.012*"fruit" + 0.011*"sauvignon"
Topic: 1 
Words: 0.025*"flavor" + 0.023*"palat" + 0.022*"finish" + 0.018*"aroma" + 0.017*"appl" + 0.015*"acid" + 0.015*"fruit" + 0.015*"note" + 0.014*"fresh" + 0.014*"white"
Topic: 2 
Words: 0.039*"wine" + 0.026*"flavor" + 0.023*"fruit" + 0.022*"tannin" + 0.017*"cherri" + 0.017*"black" + 0.015*"rich" + 0.013*"drink" + 0.013*"blackberri" + 0.012*"ripe"
Topic: 3 
Words: 0.032*"aroma" + 0.025*"palat" + 0.025*"flavor" + 0.022*"cherri" + 0.021*"finish" + 0.020*"black" + 0.017*"fruit" + 0.017*"berri" + 0.016*"tannin" + 0.015*"note"
Topic: 4 
Words: 0.073*"wine" + 0.042*"fruit" + 0.034*"acid" + 0.030*"drink" + 0.025*"flavor" + 0.023*"ripe" + 0.020*"fruiti" + 0.018*"textur" + 0.018*"rich" + 0.017*"fresh"


In [ ]:
        ########################       TOPIC MODELING VISUALIZATION             ####################

In [21]:
#Import libraries reauired for visualizing the wine topics.
from matplotlib import pyplot as plt
from wordcloud import WordCloud
import matplotlib.colors as mcolors

In [ ]:
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]

cloud = WordCloud(stopwords=gensim.parsing.preprocessing.STOPWORDS,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()